In [1]:
#Importing the required libraries

import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Reading the movie dataset

movies_data1=pd.read_csv("/Users/kshirjadiwan/Downloads/archive-2/movies_metadata.csv")

# Using the dataset upto a certain number of rows because of its large size

movies_data=movies_data1.loc[0:23000]
movies_data.head(23000)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22995,False,NaN,22000000,"[{'id': 18, 'name': 'Drama'}]",NaN,235260,tt3210686,en,Son of God,The life story of Jesus is told from his humbl...,...,2014-02-28,67800064.0,138.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Their Empire. His Kingdom.,Son of God,False,5.9,83.0
22996,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",http://www.fightville.net/,94331,tt1703232,en,Fightville,FIGHTVILLE is about the art and sport of fight...,...,2011-03-12,0.0,85.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,We Build Better Men,Fightville,False,5.8,4.0
22997,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,24202,tt0480268,en,Blind Date,"Affected by tragedy, a married couple decide t...",...,2007-05-16,0.0,80.0,[],Released,NaN,Blind Date,False,4.5,2.0
22998,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",http://www.sho.com/sho/movies/titles/97913/bli...,238972,tt0135166,en,Blind Faith,in 1957 an African-American criminal lawyer (C...,...,1998-11-23,0.0,122.0,[],Released,What would you be willing to sacrifice to keep...,Blind Faith,False,6.5,2.0


In [3]:
# Adding a column named index

listnum=list(range(0,23001,1))
movies_data.insert(0, "index", listnum, True)
movies_data.head()

,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
#Displaying the numbers of rows and colums of the dataset being used

movies_data.shape

(23001, 25)

In [5]:
#Selecting the criteria for similarity

deciders = ['genres', 'original_title', 'overview', 'tagline', 'original_language']
print(deciders)

['genres', 'original_title', 'overview', 'tagline', 'original_language']


In [6]:
#Tackling the null values of the dataset

for criteria in deciders:
    movies_data[criteria] = movies_data[criteria].fillna(' ')

In [7]:
#Combining only the selected criteria for easier computation

combined = movies_data['genres']+' '+movies_data['original_title']+' '+movies_data['overview']+' '+movies_data['tagline']+movies_data['original_language']
print(combined)

0        [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1        [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2        [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3        [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4        [{'id': 35, 'name': 'Comedy'}] Father of the B...
                               ...                        
22996    [{'id': 99, 'name': 'Documentary'}] Fightville...
22997    [{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...
22998    [{'id': 18, 'name': 'Drama'}] Blind Faith in 1...
22999    [{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...
23000    [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
Length: 23001, dtype: object


In [8]:
#Using tfidfvectorizer which transforms texts to feature vectors 

vectorizer = TfidfVectorizer()
abc = vectorizer.fit_transform(combined)

In [9]:
#Using cosine similarity to measure the similarity between vectors

coeff = cosine_similarity(abc)

In [10]:
movie_name = input(' Enter the movie name : ')

#Listing all movie titles

movie_name_list = movies_data['original_title'].tolist()

#Getting a close match for the input movie

close_match_list = difflib.get_close_matches(movie_name, movie_name_list)
match = close_match_list[0]
match_movie_index = movies_data[movies_data.original_title == match]['index'].values[0]
score = list(enumerate(coeff[match_movie_index]))

#Sorts the cosine similarity scores in descending order

movies_recommended = sorted(score, key = lambda x:x[1], reverse = True) 

print('The suggested movies are : \n')
i = 1
for movie in movies_recommended:
    j = movie[0]
    title_from_index = movies_data[movies_data.index==j]['original_title'].values[0]
    if (i<26):
        print(i, '.',title_from_index)
        i=i+1

 Enter the movie name : Interstellar
The suggested movies are : 

1 . Interstellar
2 . ほしのこえ
3 . Suburban Commando
4 . Stargate
5 . Starship Troopers
6 . Prometheus
7 . The Day the Earth Stood Still
8 . Na srebrnym globie
9 . Le Voyage dans la Lune
10 . Spacehunter: Adventures in the Forbidden Zone
11 . Man of Steel
12 . Titan A.E.
13 . Fantastic Voyage
14 . Voyage to the Bottom of the Sea
15 . Mutant Chronicles
16 . Explorers
17 . Ghost Voyage
18 . Gattaca
19 . Frank Herbert's Children of Dune
20 . This Island Earth
21 . 宇宙大戦争
22 . From the Earth to the Moon
23 . Killer Klowns from Outer Space
24 . I Married a Monster from Outer Space
25 . The Wild Blue Yonder
